## 패키지 호출

In [607]:
import os
# install konlpy, jdk, JPype
!pip install konlpy
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!pip3 install JPype1-py3

# install mecab-ko
os.chdir('/tmp/')
!curl -LO https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
!tar zxfv mecab-0.996-ko-0.9.2.tar.gz
os.chdir('/tmp/mecab-0.996-ko-0.9.2')
!./configure
!make
!make check
!make install

# install mecab-ko-dic
!apt-get install automake
os.chdir('/tmp')
!curl -LO https://bitbucket.org/eunjeon/mecab-ko-dic/downloads/mecab-ko-dic-2.1.1-20180720.tar.gz
!tar -zxvf mecab-ko-dic-2.1.1-20180720.tar.gz
os.chdir('/tmp/mecab-ko-dic-2.1.1-20180720')
!./autogen.sh
!./configure
!make
!make install

# install mecab-python
os.chdir('/content')
!git clone https://bitbucket.org/eunjeon/mecab-python-0.996.git
os.chdir('/content/mecab-python-0.996')
!python3 setup.py build
!python3 setup.py install

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1381k  100 1381k    0     0  1335k      0  0:00:01  0:00:01 --:--:-- 2283k
mecab-0.996-ko-0.9.2/
mecab-0.996-ko-0.9.2/example/
mecab-0.996-ko-0.9.2/example/example.cpp
mecab-0.996-ko-0.9.2/example/example_lattice.cpp
mecab-0.996-ko-0.9.2/example/example_lattice.c
mecab-0.996-ko-0.9.2/example/example.c
mecab-0.996-ko-0.9.2/example/thread_test.cpp
mecab-0.996-ko-0.9.2/mecab-config.in
mecab-0.996-ko-0.9.2/man/
mecab-0.996-ko-0.9.2/man/Makefile.am
mecab-0.996-ko-0.9.2/man/mecab.1
mecab-0.996-ko-0.9.2/man/Makefile.in
mecab-0.996-ko-0.9.2/mecab.iss.in
mecab-0.996-ko-0.9.2/config.guess
mecab-0.996-ko-0.9.2/README
mecab-0.996-ko-0.9.2/COPYING
mecab-0.996-ko-0.9.2/CHANGES.md
mecab-0.996-ko-0.9.2/README.md
mecab-0.996-ko-0.9.2/INSTALL
mecab-0.996-ko-0.9.2/con

In [608]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
from tqdm import tqdm_notebook

from konlpy.tag import Okt
from collections import Counter
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
import nltk
nltk.download('punkt')
import re

import os
pd.options.display.max_info_columns =200
pd.options.display.max_columns = 200
pd.options.display.max_info_rows =100
pd.options.display.max_rows = 100

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## DataFrame 전처리 함수

In [609]:
def merge_df(df):
    # 데이터 merge
#     df = total_df[0].copy()
    df.fillna(0,inplace=True)
    df.replace(0,'',inplace=True)
    df['merge'] = df['title'] + df['cat1'] #+ df['cat2'] + df['cat3']
    
    # 작업용 df로 변경
    df = df[['merge','keyword']]
    return df

In [610]:
# 태그 선별
def tag(cat):
  label_unique = 0
  for i in range(len(cat)-1):
    if i == 0 :
      label_unique = cat['label'][i] + ','
    elif i < (len(cat)-1):
      label_unique = label_unique + cat['label'][i] + ','
    else:
      label_unique = label_unique + cat['label'][i]
  label_unique = list(label_unique.split(','))

  # 공백 제거
  label_unique = [v for v in label_unique if v] 
  return(label_unique)

In [611]:
# 10번 이상 노출된 태그 list 추출
def top_tag(label_unique):
  label_unique_df = pd.DataFrame(label_unique,columns=['label_unique'])
  label_unique_df['count'] = 0
  label_unique_df = label_unique_df.groupby(['label_unique']).count().reset_index()
  label_unique_df = label_unique_df[label_unique_df['count']>10].sort_values(by='count',ascending=False)
  label_unique_df.index = range(len(label_unique_df))

  top = list(label_unique_df['label_unique'])
  return(top)

In [612]:
# label 각각 분리
def split_label(cat):
  for i in range(1,6):
      cat['label' + str(i)] = 0
  for i in range(len(cat)):
      try:
          cat.label1[i] = cat.label[i].split(',')[0]
      except: continue
      try:
          cat.label2[i] = cat.label[i].split(',')[1]
      except: continue
      try:
          cat.label3[i] = cat.label[i].split(',')[2]
      except: continue
      try:
          cat.label4[i] = cat.label[i].split(',')[3]
      except: continue
      try:
          cat.label5[i] = cat.label[i].split(',')[4]
      except: continue
  return(cat)

## 텍스트 전처리

In [613]:
def proccessed(cat):
  info=[]

  # 데이터 프레임 feature를 list로
  for i in range(len(cat)):
      info.append(cat['feature'][i])

  # feature를 각각 형태소 분리
  tokenizer = Mecab()
  nouns=[]
  for line in tqdm_notebook(info):
      nouns.append(tokenizer.nouns(line))

  # 이중 리스트로 되어 있는 텍스트를 하나의 리스트로 변형    
  test = []
  for i in range(len(nouns)):
      ## 한글자 처리
      for k in range(len(nouns[i])):
        if len(nouns[i][k]) == 1:
          nouns[i][k] = '0'

      temp=''
      for j in range(len(nouns[i])):
        if j < len(nouns[i])-1:
          temp += nouns[i][j] + ' '
        else:
          temp = temp + nouns[i][j]
      test.append(temp)
  info = test

  # 특수문자, 불용어 제거
  for i in range(len(info)):
    info[i] = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]", "", info[i])
    
  # stop_word 지정 및 적용
  stop_words = "은 는 이 가 이하 이상 차 형 용품 용 ㅍ ㅍㅍ ㅠ ㅠㅠ ㅜ ㅜㅜ ㅊ ㅊㅊ ㄸ ㅎㅎ" 
  stop_words = stop_words.split(' ') 

  for i in range(len(info)):
    word_tokens = word_tokenize(info[i]) 

    result = [] 
    for w in word_tokens: 
      if w not in stop_words: 
        result.append(w)
    info[i] = result

  # 이중 리스트로 되어 있는 텍스트를 하나의 리스트로 변형
  test = []
  for i in range(len(info)):
    temp=''
    for j in range(len(info[i])):
      if j < len(info[i])-1:
        temp += info[i][j] + ','
      else:
        temp = temp + info[i][j]
    test.append(temp)
  info = test
  return(info)

In [614]:
# 입력정보 전처리
def proccessed_input(text):
  info=text

  # feature를 각각 형태소 분리
  tokenizer = Mecab()
  nouns=[]
  for line in info:
      nouns.append(tokenizer.nouns(line))
  nouns

  test = []
  for i in range(len(nouns)):
      ## 한 글자 처리
      # for k in range(len(nouns[i])):
      #   if len(nouns[i][k]) == 1:
      #     nouns[i][k] = '0'

      temp=''
      for j in range(len(nouns[i])):
        if j < len(nouns[i])-1:
          temp += nouns[i][j] + ' '
        else:
          temp = temp + nouns[i][j]
      test.append(temp)
  info = test

  for i in range(len(info)):
    info[i] = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]", "", info[i])

  # stop_word 지정 및 적용
  stop_words = "은 는 이 가 이하 이상 차 형 용품 용 ㅍ ㅍㅍ ㅠ ㅠㅠ ㅜ ㅜㅜ ㅊ ㅊㅊ ㄸ ㅎㅎ" 
  stop_words = stop_words.split(' ') 

  for i in range(len(info)):
    word_tokens = word_tokenize(info[i]) 

    result = [] 
    for w in word_tokens: 
      if w not in stop_words: 
        result.append(w)
    info[i] = result

  # 이중 리스트로 되어 있는 텍스트를 하나의 리스트로 변형
  test = []
  for i in range(len(info)):
      temp=''
      for j in range(len(info[i])):
        if j < len(info[i])-1:
          temp += info[i][j] + ','
        else:
          temp = temp + info[i][j]
      test.append(temp)
  info = test
  info
  return(info)

## tfidf 처리

In [615]:
def tfidf(proccessed_cat):
  from sklearn.feature_extraction.text import CountVectorizer
  vect = CountVectorizer()
  countvect = vect.fit_transform(proccessed_cat)

  countvect_df = pd.DataFrame(countvect.toarray(), columns = sorted(vect.vocabulary_))

  from sklearn.feature_extraction.text import TfidfVectorizer
  vect = TfidfVectorizer(max_features = 10000)
  tfvect = vect.fit(proccessed_cat)

  tfidv_df = pd.DataFrame(tfvect.transform(proccessed_cat).toarray(), columns = sorted(vect.vocabulary_))
  return(tfidv_df)

In [616]:
def make_matrix(tag,cat):
  total_lab = pd.DataFrame()
  for label in tqdm(tag):
    temp = pd.DataFrame()
    for i in range(len(cat)):
        if cat['label1'][i] == label:
            temp = pd.concat([temp, cat.iloc[[i]]])
        elif cat['label2'][i] == label:
            temp = pd.concat([temp, cat.iloc[[i]]])
        elif cat['label3'][i] == label:
            temp = pd.concat([temp, cat.iloc[[i]]])
        elif cat['label4'][i] == label:
            temp = pd.concat([temp, cat.iloc[[i]]])
        elif cat['label5'][i] == label:
            temp = pd.concat([temp, cat.iloc[[i]]])
    temp_tfidf = tfidf(temp['feature'])
    temp_tfidf['label'] = label
    temp_tfidf = temp_tfidf.groupby('label').mean().reset_index()
    total_lab = pd.concat([total_lab,temp_tfidf])
  total_lab = total_lab.fillna(0)
  return(total_lab)
  

## 태그 추출 함수 선언

In [617]:
# title = [input('제목을 입력해주세요 : ')]
def find_tag(title, cat_matrix):
  test = proccessed_input(list(title))
  test = test[0].split(',')

  test_input = []
  for i in test:
    if i in cat_matrix.columns:
      test_input.append(i)
  test_input.append('label')

  # 해당 카테고리에 적용
  test_matrix = cat_matrix[test_input]
  test_matrix

  # 상위 top5 태그 추출
  test_matrix['target'] = 0
  for i in range(len(test_input)-1):
    test_matrix['target'] += test_matrix[test_input[i]] 
  test_matrix['target'] = test_matrix['target'] / (len(test_input)-1)
  test_matrix = test_matrix[['label','target']].sort_values(by='target', ascending=False)[:5]
  return(test_matrix)

## 수기 전처리부분

In [618]:
# 작업 목록 df로 불러오기
query_list = ['갤럭시','구두','노트북','데스크탑','마우스','모자','바지','상의','셔츠','스니커즈','시계','아이폰','안경','에어컨',
              '에어팟','오토바이','원피스','전자제품','점퍼','중고차','쥬얼리','지갑','치마','카메라','코트','키보드','티비']

total_df = []
for i in range(len(query_list)):
    total_df.append(pd.read_csv(f'/content/drive/MyDrive/Colab Notebooks/1조/태그크롤링데이터/{query_list[i]}.csv'))

In [619]:
lab = total_df[2]

In [620]:
lab = lab[(lab['cat1']=='디지털/가전')]
lab = lab[(lab['cat2']=='PC/노트북')]
lab.index=range(len(lab))
lab

,product_id,title,keyword,cat1,cat2,cat3,view,image
0,171860206,[전문]A급 8세대 1050 램12기가 옵테인500 한성 게이밍노트북,"대전노트북,게이밍,노트북,msi,보스몬스터",디지털/가전,PC/노트북,노트북/넷북,677,https://media.bunjang.co.kr/product/171860206_...
1,171044889,[전문]A급 i7 램16기가 ssd듀얼보스몬스터 게이밍노트북,"대전노트북,msi,한성노트북,게이밍,노트북",디지털/가전,PC/노트북,노트북/넷북,719,https://media.bunjang.co.kr/product/171044889_...
2,152053798,[전문]17인치 S급 풀박 i7 램16기가 ssd512 게이밍노트북,"대전노트북,msi,게이밍,노트북,1660ti",디지털/가전,PC/노트북,노트북/넷북,2892,https://media.bunjang.co.kr/product/152053798_...
3,152043062,[전문]A급 배터리신품 초슬림 i3 램8기가 ssd 240 hp노트북,"대전노트북,15인치,hp 노트북,인강용",디지털/가전,PC/노트북,노트북/넷북,1785,https://media.bunjang.co.kr/product/152043062_...
4,174538693,LG 노트북 13.3인치 팬티엄,"엘지,엘지그램,노트북,엘지노트북,삼성노트북",디지털/가전,PC/노트북,노트북/넷북,27,https://media.bunjang.co.kr/product/174538693_...
...,...,...,...,...,...,...,...,...
4873,170928907,삼성 노트북 실사용 5회미만,NaN,디지털/가전,PC/노트북,노트북/넷북,105,https://media.bunjang.co.kr/product/170928907_...
4874,172834182,에이서 노트북,"에이서노트북,에이서,acer",디지털/가전,PC/노트북,노트북/넷북,49,https://media.bunjang.co.kr/product/172834182_...
4875,173905950,HP 15.6인치 게이밍 터치스크린 노트북 판매합니다,"터치스크린노트북,hp게이밍노트북,hp터치스마트,노트북,게이밍노트북",디지털/가전,PC/노트북,노트북/넷북,62,https://media.bunjang.co.kr/product/173905950_...
4876,172847961,레노버 노트북,NaN,디지털/가전,PC/노트북,노트북/넷북,218,https://media.bunjang.co.kr/product/172847961_...


## 한번에 구현(1차 카테고리 수기 전처리 이후)
태그 전처리 및 전체 feature 전처리

In [621]:
# 데이터 처리를 위한 df 설정
lab = merge_df(lab)
lab.rename(columns={'merge':'feature', 'keyword' : 'label'}, inplace=True)
lab = split_label(lab)
lab

# feature 전처리
pro_lab = proccessed(lab)
lab['feature'] = pro_lab

# 태그 전처리, 10번 이상 노출된 태그만 추출
label_unique = tag(lab)
tag_list = top_tag(label_unique)

# labtop matrix 생성
lab_matrix = make_matrix(tag_list,lab)
lab_matrix

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  0%|          | 0/4878 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


  0%|          | 0/156 [00:00<?, ?it/s]

,label,가격,가능,가방,가성,가입,가전,가정,가죽,가지,각인,각종,강력,강용,개발자,개봉,개인,갤러,갤럭시,거래,거스,거치,거치대,검색,게이밍,게임,게임기,견적,견적도,겸용,경기도,경량,경량급,경북,고가,고급,고사,고성능,고속,고양시,고장,고전,고해상도,고화질,골드,공간,공대,공짜,광안,광주광역시,교신,교체,교환,구매,구성,구입,구형,국내,군산,그라운드,그라운즈,그래픽,그램,그레이,글위,급매,기가,기가바이트,기간,기기,기능,기본,기스,기어,기타,꾸미기,나이트,남음,내림,내부,내용,내장,네오,네이비,넥스트,넷북,넷플릭스,노바,노보,노우,노크,노트,노트북,니다,니트,다크,단순,단자,단조,단종,...,배낭,변신,보헤미안,브리프,소나이,숄더백,스트라이크,스프링거,시에라,시트,의자,쥬얼,캐리어,크기,크럼플러,팔찌,미공개,경기,고양,덕양,마포,무한,서대문,용산,프린터,선입,여성,우대,유니,과제,대학,인민,테클라,부풍,젠더,뤼팽,아르센,골드색,조립식,전체,브릿지,샌디,우체국,제노바,러닝,흠집,닌텐도,등산,레저,자전거,전동,졸리,캠핑,타임,피싱,가로,각도,강의,교정,단거,럭스,레인,맥컬리,분산,알류,에이트,우드,이지,자세,조절,크리,클레버,타워,파우트,폐업,한국,후면,히트,미사,유초,장난감,멜로,오브젝트,플래닛,광시,샌드,트랜,스마트폰,북백,라이더,악세,어반,사은품,덮개,걸리,수원,탑동,요번,크로마,개별
0,노트북,0.008327,0.007920,0.007535,0.006059,0.000256,0.172193,0.000208,0.000295,0.000169,0.00037,0.000177,0.000305,0.005415,0.000173,0.002387,0.001456,0.000260,0.026759,0.000244,0.000418,0.000197,0.003402,0.000614,0.044570,0.003345,0.000221,0.00028,0.000302,0.000794,0.000222,0.003904,0.000259,0.000219,0.000339,0.004644,0.006080,0.002582,0.000226,0.000222,0.000778,0.000305,0.000401,0.000266,0.004486,0.000396,0.000208,0.00023,0.000315,0.000305,0.001309,0.001358,0.001321,0.002867,0.000243,0.000733,0.000685,0.000667,0.000218,0.000172,0.000354,0.005917,0.038544,0.007857,0.000309,0.000834,0.007656,0.002370,0.00062,0.000641,0.000226,0.000838,0.000332,0.001122,0.000249,0.000198,0.000435,0.00062,0.006973,0.000217,0.000455,0.000225,0.000372,0.000784,0.001283,0.001635,0.000439,0.000353,0.000644,0.000714,0.000264,0.000276,0.169932,0.004587,0.000762,0.000397,0.000229,0.00017,0.000197,0.000309,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,삼성노트북,0.004362,0.007233,0.003650,0.009159,0.000000,0.215849,0.000876,0.000000,0.000000,0.00000,0.000000,0.001374,0.014762,0.000000,0.004240,0.000000,0.001079,0.045105,0.000000,0.000000,0.000000,0.000000,0.001262,0.015836,0.003496,0.000858,0.00000,0.000000,0.000000,0.000000,0.005457,0.000000,0.000000,0.001461,0.001838,0.004931,0.002619,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000930,0.000000,0.00000,0.000000,0.000000,0.002625,0.001394,0.000000,0.000978,0.000000,0.000930,0.001355,0.002010,0.000000,0.000000,0.000000,0.006118,0.007260,0.000000,0.000000,0.000000,0.007294,0.000000,0.00000,0.000000,0.000000,0.000000,0.001529,0.000000,0.000000,0.000000,0.000000,0.00000,0.003613,0.000000,0.000000,0.000000,0.000000,0.000997,0.000000,0.000000,0.002395,0.000000,0.000000,0.000000,0.000000,0.000000,0.230344,0.007799,0.000000,0.000000,0.001058,0.00000,0.000000,0.001367,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,삼성,0.007968,0.006570,0.009578,0.008186,0.000000,0.228542,0.000000,0.001691,0.000000,0.00000,0.000000,0.000000,0.012881,0.000000,0.001411,0.000000,0.000000,0.046694,0.000000,0.000000,0.000000,0.001384,0.001788,0.024161,0.004035,0.000000,0.00000,0.000000,0.000000,0.000000,0.003697,0.001569,0.000000,0.000000,0.004603,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.002564,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001374,0.000000,0.001376,0.000000,0.000000,0.000000,0.010274,0.013011,0.000000,0.000000,0.002564,0.003922,0.000000,0.00000,0.002144,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.004104,0.000000,0.000000

In [622]:
lab_matrix

,label,가격,가능,가방,가성,가입,가전,가정,가죽,가지,각인,각종,강력,강용,개발자,개봉,개인,갤러,갤럭시,거래,거스,거치,거치대,검색,게이밍,게임,게임기,견적,견적도,겸용,경기도,경량,경량급,경북,고가,고급,고사,고성능,고속,고양시,고장,고전,고해상도,고화질,골드,공간,공대,공짜,광안,광주광역시,교신,교체,교환,구매,구성,구입,구형,국내,군산,그라운드,그라운즈,그래픽,그램,그레이,글위,급매,기가,기가바이트,기간,기기,기능,기본,기스,기어,기타,꾸미기,나이트,남음,내림,내부,내용,내장,네오,네이비,넥스트,넷북,넷플릭스,노바,노보,노우,노크,노트,노트북,니다,니트,다크,단순,단자,단조,단종,...,배낭,변신,보헤미안,브리프,소나이,숄더백,스트라이크,스프링거,시에라,시트,의자,쥬얼,캐리어,크기,크럼플러,팔찌,미공개,경기,고양,덕양,마포,무한,서대문,용산,프린터,선입,여성,우대,유니,과제,대학,인민,테클라,부풍,젠더,뤼팽,아르센,골드색,조립식,전체,브릿지,샌디,우체국,제노바,러닝,흠집,닌텐도,등산,레저,자전거,전동,졸리,캠핑,타임,피싱,가로,각도,강의,교정,단거,럭스,레인,맥컬리,분산,알류,에이트,우드,이지,자세,조절,크리,클레버,타워,파우트,폐업,한국,후면,히트,미사,유초,장난감,멜로,오브젝트,플래닛,광시,샌드,트랜,스마트폰,북백,라이더,악세,어반,사은품,덮개,걸리,수원,탑동,요번,크로마,개별
0,노트북,0.008327,0.007920,0.007535,0.006059,0.000256,0.172193,0.000208,0.000295,0.000169,0.00037,0.000177,0.000305,0.005415,0.000173,0.002387,0.001456,0.000260,0.026759,0.000244,0.000418,0.000197,0.003402,0.000614,0.044570,0.003345,0.000221,0.00028,0.000302,0.000794,0.000222,0.003904,0.000259,0.000219,0.000339,0.004644,0.006080,0.002582,0.000226,0.000222,0.000778,0.000305,0.000401,0.000266,0.004486,0.000396,0.000208,0.00023,0.000315,0.000305,0.001309,0.001358,0.001321,0.002867,0.000243,0.000733,0.000685,0.000667,0.000218,0.000172,0.000354,0.005917,0.038544,0.007857,0.000309,0.000834,0.007656,0.002370,0.00062,0.000641,0.000226,0.000838,0.000332,0.001122,0.000249,0.000198,0.000435,0.00062,0.006973,0.000217,0.000455,0.000225,0.000372,0.000784,0.001283,0.001635,0.000439,0.000353,0.000644,0.000714,0.000264,0.000276,0.169932,0.004587,0.000762,0.000397,0.000229,0.00017,0.000197,0.000309,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,삼성노트북,0.004362,0.007233,0.003650,0.009159,0.000000,0.215849,0.000876,0.000000,0.000000,0.00000,0.000000,0.001374,0.014762,0.000000,0.004240,0.000000,0.001079,0.045105,0.000000,0.000000,0.000000,0.000000,0.001262,0.015836,0.003496,0.000858,0.00000,0.000000,0.000000,0.000000,0.005457,0.000000,0.000000,0.001461,0.001838,0.004931,0.002619,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000930,0.000000,0.00000,0.000000,0.000000,0.002625,0.001394,0.000000,0.000978,0.000000,0.000930,0.001355,0.002010,0.000000,0.000000,0.000000,0.006118,0.007260,0.000000,0.000000,0.000000,0.007294,0.000000,0.00000,0.000000,0.000000,0.000000,0.001529,0.000000,0.000000,0.000000,0.000000,0.00000,0.003613,0.000000,0.000000,0.000000,0.000000,0.000997,0.000000,0.000000,0.002395,0.000000,0.000000,0.000000,0.000000,0.000000,0.230344,0.007799,0.000000,0.000000,0.001058,0.00000,0.000000,0.001367,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,삼성,0.007968,0.006570,0.009578,0.008186,0.000000,0.228542,0.000000,0.001691,0.000000,0.00000,0.000000,0.000000,0.012881,0.000000,0.001411,0.000000,0.000000,0.046694,0.000000,0.000000,0.000000,0.001384,0.001788,0.024161,0.004035,0.000000,0.00000,0.000000,0.000000,0.000000,0.003697,0.001569,0.000000,0.000000,0.004603,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.002564,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001374,0.000000,0.001376,0.000000,0.000000,0.000000,0.010274,0.013011,0.000000,0.000000,0.002564,0.003922,0.000000,0.00000,0.002144,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.004104,0.000000,0.000000

## 실제 데이터 입력 후 구현

In [625]:
title = [input('제목을 입력해주세요 : ')]
tag = find_tag(title, cat_matrix = lab_matrix)

제목을 입력해주세요 : lg그램 i7코어 중고가로 팝닌다


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

In [626]:
tag

,label,target
0,LG전자,0.200380
0,gram,0.150458
0,그램노트북,0.141891
0,lg그램,0.130941
0,엘지그램,0.116384


## 다른 카테고리도 시도(수기 전처리 미실시)

In [ ]:
gal = total_df[0]
gal

In [ ]:
# 데이터 처리를 위한 df 설정
gal = merge_df(gal)
gal.rename(columns={'merge':'feature', 'keyword' : 'label'}, inplace=True)
# 태그 분배
gal = split_label(gal)

# feature 전처리
pro_gal = proccessed(gal)
gal['feature'] = pro_gal

# 태그 전처리, 10번 이상 노출된 태그만 추출
label_unique = tag(gal)
tag_list = top_tag(label_unique)

# 갤럭시 matrix 생성
gal_matrix = make_matrix(tag_list,gal)